# 自動調參數
### 參數使用
```
class sklearn.model_selection.GridSearchCV(estimator, param_grid, scoring=None, fit_params=None, n_jobs=1, iid=True, refit=True, cv=None, verbose=0, pre_dispatch=‘2*n_jobs’, error_score=’raise’, return_train_score=’warn’)

estimator：所使用的分類器，比如：estimator=RandomForestClassifier(min_samples_split=100, min_samples_leaf=20, max_depth=8, max_features=‘sqrt‘, random_state=10)，並且傳入除需要確定最佳的參數之外的其他參數。每個分類器都需要一個scoring參數或者score方法。
param_grid：值為字典或列表，即需要最優化的參數的取值，param_grid =param_test1，param_test1 = {‘n_estimators‘:range(10,71,10)}
scoring：準確評價標準，默認為None（使用estimator的誤差估計函數），這時需要使用score函數；或者如scoring=‘roc_auc‘，根據所選模型不同，評價準則不同。
cv：交叉驗證參數，默認為None
refit：默認為True，程序將會以交叉驗證訓練集得到的最佳參數，重新對所有可用的訓練集與測試集進行，作為最終用於性能評估的最佳模型參數。即在搜索參數結束後，用最佳參數結果再次fit一遍全部數據集。
iid:默認True,為True時，默認為各個樣本fold概率分布一致，誤差估計為所有樣本之和，而非各個fold的平均。

verbose：日誌冗長度，int：冗長度，0：不輸出訓練過程，1：偶爾輸出，>1：對每個子模型都輸出。

n_jobs: 並行數，int：個數,-1：跟CPU核數一致, 1:默認值。

pre_dispatch：指定總共分發的並行任務數。當n_jobs大於1時，數據將在每個運行點進行復制，這可能導致OOM，而設置pre_dispatch參數，則可以預先劃分總共的job數量，使數據最多被復制pre_dispatch次，進行預測的常用方法和屬性

grid.fit()：運行網格搜索

grid_scores_：給出不同參數情況下的評價結果

best_params_：描述了已取得最佳結果的參數的組合

best_score_：成員提供優化過程期間觀察到的最好的評分
```
### 屬性方法：

* grid.fit( train_x, train_y )：運行網格搜索
* grid_scores_：給出不同參數情況下的評價結果
* best_params_：描述了已取得最佳結果的參數的組合
* best_score_：成員提供優化過程期間觀察到的最好的評分
[GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html)

# Progress bar for models fitting

> conda install keras-tqdm

[tqdm](https://github.com/bstriner/keras-tqdm)


In [3]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split,GridSearchCV#自動調參
from sklearn.feature_extraction import DictVectorizer
import pandas as pd

import warnings
warnings.filterwarnings('ignore')
#from keras_tqdm import TQDMCallback

datapath = "data/titanic_data.csv"

def RandomForestTest():
    data = pd.read_csv(datapath)
    X = data[["Pclass","Sex","Age"]]
    Y = data["Survived"]

    X["Age"].fillna(X["Age"].mean(),inplace=True)

    X_train,X_test,y_train,y_test = train_test_split(X,Y,test_size=0.25)

    DV = DictVectorizer(sparse=False)
    X_train = DV.fit_transform(X_train.to_dict(orient="records"))
    X_test = DV.transform(X_test.to_dict(orient="records"))
    
    # 實例化隨機森林分類器
    RF = RandomForestClassifier(verbose=1)#verbose=True #verbose=0, callbacks=[TQDMCallback()]
    
    # 測試超參數優化
    RF_param = {"n_estimators":[100,300],"max_depth":[5]}
    
    print(type(GridSearchCV(RF,param_grid=RF_param,cv=5)))
    GSCV = (GridSearchCV(RF,param_grid=RF_param,cv=5))
    GSCV.fit(X_train,y_train)
    
    print("accuracy score",GSCV.score(X_test,y_test))
    print("The best model param :",GSCV.best_params_)



In [4]:
RandomForestTest()

<class 'sklearn.model_selection._search.GridSearchCV'>


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_j

accuracy score 0.7802690582959642
The best model param : {'max_depth': 5, 'n_estimators': 100}


[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished


In [6]:
from time import sleep
from tqdm import tqdm,trange
pbar = tqdm(total=10000)

for i in range(10000):
    pbar.set_description('Processing %d'%i)
    pbar.update(10)
    sleep(0.001)




  0%|                                                                                        | 0/10000 [00:00<?, ?it/s]

Processing 0:   0%|                                                                          | 0/10000 [00:00<?, ?it/s]

Processing 1:   0%|                                                                | 10/10000 [00:00<00:16, 621.75it/s]

Processing 2:   0%|▏                                                               | 20/10000 [00:00<00:15, 633.12it/s]

Processing 3:   0%|▏                                                               | 30/10000 [00:00<00:15, 630.35it/s]

Processing 4:   0%|▎                                                               | 40/10000 [00:00<00:15, 637.31it/s]

Processing 5:   0%|▎                                                               | 50/10000 [00:00<00:15, 638.39it/s]

Processing 6:   1%|▍                                                               | 60/10000 [00:00<00:15, 636.09it/s]

Processing 7:   1%|▍          

Processing 56:   6%|███▍                                                          | 560/10000 [00:00<00:14, 639.57it/s]

Processing 57:   6%|███▌                                                          | 570/10000 [00:00<00:14, 639.57it/s]

Processing 57:   6%|███▌                                                          | 580/10000 [00:00<00:14, 632.90it/s]

Processing 58:   6%|███▌                                                          | 580/10000 [00:00<00:14, 632.90it/s]

Processing 59:   6%|███▋                                                          | 590/10000 [00:00<00:14, 632.90it/s]

Processing 60:   6%|███▋                                                          | 600/10000 [00:00<00:14, 632.90it/s]

Processing 61:   6%|███▊                                                          | 610/10000 [00:00<00:14, 632.90it/s]

Processing 62:   6%|███▊                                                          | 620/10000 [00:00<00:14, 632.90it/s]

Processing 63:   6%|███▉        

Processing 112:  11%|██████▋                                                     | 1120/10000 [00:01<00:13, 643.72it/s]

Processing 113:  11%|██████▊                                                     | 1130/10000 [00:01<00:13, 643.72it/s]

Processing 113:  11%|██████▊                                                     | 1140/10000 [00:01<00:13, 642.83it/s]

Processing 114:  11%|██████▊                                                     | 1140/10000 [00:01<00:13, 642.83it/s]

Processing 115:  12%|██████▉                                                     | 1150/10000 [00:01<00:13, 642.83it/s]

Processing 116:  12%|██████▉                                                     | 1160/10000 [00:01<00:13, 642.83it/s]

Processing 117:  12%|███████                                                     | 1170/10000 [00:01<00:13, 642.83it/s]

Processing 118:  12%|███████                                                     | 1180/10000 [00:01<00:13, 642.83it/s]

Processing 119:  12%|███████▏   

Processing 168:  17%|██████████                                                  | 1680/10000 [00:02<00:12, 646.05it/s]

Processing 169:  17%|██████████▏                                                 | 1690/10000 [00:02<00:12, 646.05it/s]

Processing 169:  17%|██████████▏                                                 | 1700/10000 [00:02<00:12, 644.41it/s]

Processing 170:  17%|██████████▏                                                 | 1700/10000 [00:02<00:12, 644.41it/s]

Processing 171:  17%|██████████▎                                                 | 1710/10000 [00:02<00:12, 644.41it/s]

Processing 172:  17%|██████████▎                                                 | 1720/10000 [00:02<00:12, 644.41it/s]

Processing 173:  17%|██████████▍                                                 | 1730/10000 [00:02<00:12, 644.41it/s]

Processing 174:  17%|██████████▍                                                 | 1740/10000 [00:02<00:12, 644.41it/s]

Processing 175:  18%|██████████▌

Processing 224:  22%|█████████████▍                                              | 2240/10000 [00:03<00:12, 646.06it/s]

Processing 225:  22%|█████████████▌                                              | 2250/10000 [00:03<00:11, 646.06it/s]

Processing 225:  23%|█████████████▌                                              | 2260/10000 [00:03<00:11, 649.24it/s]

Processing 226:  23%|█████████████▌                                              | 2260/10000 [00:03<00:11, 649.24it/s]

Processing 227:  23%|█████████████▌                                              | 2270/10000 [00:03<00:11, 649.24it/s]

Processing 228:  23%|█████████████▋                                              | 2280/10000 [00:03<00:11, 649.24it/s]

Processing 229:  23%|█████████████▋                                              | 2290/10000 [00:03<00:11, 649.24it/s]

Processing 230:  23%|█████████████▊                                              | 2300/10000 [00:03<00:11, 649.24it/s]

Processing 231:  23%|███████████

...

Processing 2734: : 27350it [00:42, 629.77it/s]

Processing 2735: : 27350it [00:42, 629.77it/s]

Processing 2736: : 27360it [00:42, 629.77it/s]

Processing 2737: : 27370it [00:42, 629.77it/s]

Processing 2738: : 27380it [00:42, 629.77it/s]

Processing 2739: : 27390it [00:43, 629.77it/s]

Processing 2740: : 27400it [00:43, 629.77it/s]

Processing 2741: : 27410it [00:43, 629.77it/s]

Processing 2741: : 27420it [00:43, 628.28it/s]

Processing 2742: : 27420it [00:43, 628.28it/s]

Processing 2743: : 27430it [00:43, 628.28it/s]

Processing 2744: : 27440it [00:43, 628.28it/s]

Processing 2745: : 27450it [00:43, 628.28it/s]

Processing 2746: : 27460it [00:43, 628.28it/s]

Processing 2747: : 27470it [00:43, 628.28it/s]

Processing 2748: : 27480it [00:43, 628.28it/s]

Processing 2748: : 27490it [00:43, 632.35it/s]

Processing 2749: : 27490it [00:43, 632.35it/s]

Processing 2750: : 27500it [00:43, 632.35it/s]

Processing 2751: : 27510it [00:43, 632.35it/s]

Processing 2752: : 27520it [00:43, 632.3

Processing 2867: : 28670it [00:45, 631.37it/s]

Processing 2867: : 28680it [00:45, 629.40it/s]

Processing 2868: : 28680it [00:45, 629.40it/s]

Processing 2869: : 28690it [00:45, 629.40it/s]

Processing 2870: : 28700it [00:45, 629.40it/s]

Processing 2871: : 28710it [00:45, 629.40it/s]

Processing 2872: : 28720it [00:45, 629.40it/s]

Processing 2873: : 28730it [00:45, 629.40it/s]

Processing 2874: : 28740it [00:45, 629.40it/s]

Processing 2874: : 28750it [00:45, 628.03it/s]

Processing 2875: : 28750it [00:45, 628.03it/s]

Processing 2876: : 28760it [00:45, 628.03it/s]

Processing 2877: : 28770it [00:45, 628.03it/s]

Processing 2878: : 28780it [00:45, 628.03it/s]

Processing 2879: : 28790it [00:45, 628.03it/s]

Processing 2880: : 28800it [00:45, 628.03it/s]

Processing 2881: : 28810it [00:45, 628.03it/s]

Processing 2881: : 28820it [00:45, 627.08it/s]

Processing 2882: : 28820it [00:45, 627.08it/s]

Processing 2883: : 28830it [00:45, 627.08it/s]

Processing 2884: : 28840it [00:45, 627.0

Processing 2999: : 29990it [00:47, 629.99it/s]

Processing 3000: : 30000it [00:47, 629.99it/s]

Processing 3000: : 30010it [00:47, 628.44it/s]

Processing 3001: : 30010it [00:47, 628.44it/s]

Processing 3002: : 30020it [00:47, 628.44it/s]

Processing 3003: : 30030it [00:47, 628.44it/s]

Processing 3004: : 30040it [00:47, 628.44it/s]

Processing 3005: : 30050it [00:47, 628.44it/s]

Processing 3006: : 30060it [00:47, 628.44it/s]

Processing 3007: : 30070it [00:47, 628.44it/s]

Processing 3007: : 30080it [00:47, 627.36it/s]

Processing 3008: : 30080it [00:47, 627.36it/s]

Processing 3009: : 30090it [00:47, 627.36it/s]

Processing 3010: : 30100it [00:47, 627.36it/s]

Processing 3011: : 30110it [00:47, 627.36it/s]

Processing 3012: : 30120it [00:47, 627.36it/s]

Processing 3013: : 30130it [00:47, 627.36it/s]

Processing 3014: : 30140it [00:47, 627.36it/s]

Processing 3014: : 30150it [00:47, 626.61it/s]

Processing 3015: : 30150it [00:47, 626.61it/s]

Processing 3016: : 30160it [00:47, 626.6

Processing 3131: : 31310it [00:49, 627.91it/s]

Processing 3132: : 31320it [00:49, 627.91it/s]

Processing 3133: : 31330it [00:49, 627.91it/s]

Processing 3133: : 31340it [00:49, 628.68it/s]

Processing 3134: : 31340it [00:49, 628.68it/s]

Processing 3135: : 31350it [00:49, 628.68it/s]

Processing 3136: : 31360it [00:49, 628.68it/s]

Processing 3137: : 31370it [00:49, 628.68it/s]

Processing 3138: : 31380it [00:49, 628.68it/s]

Processing 3139: : 31390it [00:49, 628.68it/s]

Processing 3140: : 31400it [00:49, 628.68it/s]

Processing 3140: : 31410it [00:49, 629.22it/s]

Processing 3141: : 31410it [00:49, 629.22it/s]

Processing 3142: : 31420it [00:49, 629.22it/s]

Processing 3143: : 31430it [00:49, 629.22it/s]

Processing 3144: : 31440it [00:49, 629.22it/s]

Processing 3145: : 31450it [00:49, 629.22it/s]

Processing 3146: : 31460it [00:49, 629.22it/s]

Processing 3147: : 31470it [00:49, 629.22it/s]

Processing 3147: : 31480it [00:49, 631.30it/s]

Processing 3148: : 31480it [00:49, 631.3

KeyboardInterrupt: 